In [47]:
#libraries
import os
import pandas as pd
from datetime import datetime as dt
from datetime import time as t
from dateutil import relativedelta
import numpy as np
from IPython.display import display

In [48]:
# source
user_credential = r'/Users/dinhhoang.nguyen.CONCENTRIX/OneDrive - Concentrix Corporation\WFM-internal'

In [49]:
# Schedule
schedule = pd.read_json(os.path.join(r'C:', user_credential, r'DB\filejson\schedule_full.json'))

In [50]:
# IMPORT TL
tl = pd.read_json(os.path.join(r'C:', user_credential, r'DB\filejson\tl.json'))

In [51]:
# Add master roster
# IMPORT MASTER ROSTER
masterroster = pd.read_json(os.path.join(r'C:', user_credential, r'DB\filejson\masterroster.json'))
masterroster['Employee_ID'] = masterroster['Employee_ID'].astype("Int64")

In [52]:
schedule = schedule[['Emp ID', 'Date', 'Shift', 'LOB']]
# Define shift
present_list = [
'0400-1300', 
'0600-1500',
'0700-1600',
'0800-1700',
'0900-1800',
'1000-1900',
'1100-2000',
'1200-2100',
'1300-2200',
'1400-2300',
'2000-0500',
'2100-0600',
'2200-0700'
]

pl_list = [
    'AL',
    'CO'
]
upl_list = [
    'UPL'
]
schedule['Shift_definition'] = schedule.apply(lambda x: 'Present' if x['Shift'] in present_list else 'Planned leave' if x['Shift'] in pl_list else 'Unplanned leave' if x['Shift'] in upl_list else "", axis=1)

In [53]:
# Attrition
termination = pd.read_excel(os.path.join(r'C:', user_credential, r'Data/Workdayattrition/attrition.xlsx'), sheet_name='Termination Dump')
termination = termination.loc[(termination['MSA'] == '5585 - Bookingcom - BOOKINGCOMVN')&(termination['JOB_ROLE'] == 'Agent')&(termination['CITY'] == 'Ho Chi Minh')][['EMPLOYEE_ID', 'LWD', 'Termination Reason']]

In [54]:
resignation = pd.read_excel(os.path.join(r'C:', user_credential, r'Data/Workdayattrition/attrition.xlsx'), sheet_name='Resignation')
resignation = resignation.loc[(resignation['Job Family']=='Contact Center')&(resignation['MSA Client']=='Bookingcom')&(resignation['Country']=='Vietnam')][['Employee ID', 'Proposed Termination Date', 'Resignation Primary Reason']]
resignation = resignation.rename(columns={'Employee ID': 'EMPLOYEE_ID', 'Proposed Termination Date': 'LWD', 'Resignation Primary Reason': 'Resignation Reason'})


In [55]:
# Attrition
transfer = pd.read_excel(os.path.join(r'C:', user_credential, r'Data\Transfer/transfer.xlsx'), sheet_name='Sheet1')
transfer = transfer[['EID', 'LWD', 'Employee Status', 'Remarks']]

In [56]:
agent_consolidate = pd.merge(termination, transfer, left_on = ['EMPLOYEE_ID', 'LWD'], right_on = ['EID', "LWD"], how='outer') # merge trmination and transfer
# agent_consolidate['Employee_ID'] = agent_consolidate.apply(lambda x: x['EMPLOYEE_ID'] if pd.isnull(x['EID']) else x['EID'], axis=1)

agent_consolidate = pd.merge(agent_consolidate, resignation, on = ['EMPLOYEE_ID', 'LWD'], how='outer')
agent_consolidate['Employee_ID'] = agent_consolidate.apply(lambda x: x['EMPLOYEE_ID'] if pd.isnull(x['EID']) else x['EID'], axis=1)

agent_consolidate['Termination Reason'] = agent_consolidate.apply(lambda x: x['Termination Reason'] if pd.notnull(x['Termination Reason']) else x['Remarks'] if pd.notnull(x['Remarks']) else x['Resignation Reason'], axis=1)
agent_consolidate = agent_consolidate.drop(columns={'EID', 'EMPLOYEE_ID', 'Employee Status', 'Remarks', 'Resignation Reason'})

In [57]:
schedule = pd.merge(schedule, agent_consolidate, left_on = ['Emp ID', 'Date'], right_on = ['Employee_ID', 'LWD'], how = 'outer')
schedule['Employee_ID'] = schedule.apply(lambda x: x['Emp ID'] if pd.isnull(x['Employee_ID']) else x['Employee_ID'], axis=1)
schedule['Date'] = schedule.apply(lambda x: x['Date'] if pd.isnull(x['LWD']) else x['LWD'], axis=1)

schedule = schedule.drop(columns={'Emp ID', 'LWD'})

In [58]:
# Add TL
schedule = pd.merge(schedule, tl, left_on='Employee_ID', right_on = 'Emp ID', how='left')
schedule = schedule.drop(columns={'Emp ID'})

In [59]:
# schedule['Termination/Transfer'] = schedule.apply(lambda x: x['Termination Reason'] if pd.isnull(x['Employee Status']) else x['Employee Status'], axis=1)

In [60]:
# schedule = schedule.drop(columns={'Termination Reason', 'Employee Status'})

In [61]:
# schedule = schedule[['Employee_ID', 'Date', 'LOB', 'Supervisor','Shift', 'Shift_definition', 'Termination/Transfer', 'Remarks']]

In [62]:
# Export
schedule.to_csv(r'C:\Users\dinhhoang.nguyen.CONCENTRIX\Concentrix Corporation\Tung Quan Le - BKN\HC_Attrition\[BcomDB] Headcount_attrition.csv', index=False)